In [4]:
import pickle as pkl
import os
import numpy as np

In [2]:
def load_pickle(filename):
    with open(filename, 'rb') as pkl_file:
        data = pkl.load(pkl_file)
    return data

In [5]:
data_dir = (os.path.join(os.path.abspath('..'), 'data'))
data_dir

'c:\\Users\\nafta\\Desktop\\Project\\TweetAnalyzer.git\\backend\\latest_bgsrd\\data'

In [66]:
df_text = load_pickle(os.path.join(data_dir, 'df_data.pkl'))
df_text

,text,label,type
0,RT @NCIDirector: This is a helpful #COVID19 re...,1,train
1,Two positive cases of #COVID19 detected in Ind...,1,train
2,"RT @Kevin_Shipp: Expert who created the ""US Bi...",0,train
3,NEW: China reported no new confirmed cases of ...,1,train
4,"In a span of 12 hours, the @MoHFW_INDIA report...",1,train
...,...,...,...
395,Feb 27 Update: Canada’s 13th case of #COVID19 ...,1,test
396,@Asadpxki Nope ebola is still an epidemic espe...,0,test
397,RT @CDCgov: Protect yourself and others from #...,1,test
398,RT @9GAG: And don't forget to protect your eye...,0,test


In [11]:
df_word_word = load_pickle(os.path.join(data_dir,'word_word_edges.pkl'))
df_word_word

[('-', '//tco/fdk82mhiaj', {'weight': 2.3862841119626634}),
 ('-', '01', {'weight': 2.3862841119626634}),
 ('-', '106000', {'weight': 0.440373963459175}),
 ('-', '18', {'weight': 0.08369901979635494}),
 ('-', '2030', {'weight': 1.2876718234784956}),
 ('-', '24h', {'weight': 0.440373963459175}),
 ('-', '300', {'weight': 2.134969683708035}),
 ('-', '3⃣', {'weight': 0.1890595353622106}),
 ('-', 'ability', {'weight': 1.2876718234784956}),
 ('-', 'accept', {'weight': 1.693136931494689}),
 ('-', 'access', {'weight': 0.4403739634591752}),
 ('-', 'actively', {'weight': 2.2039625551871027}),
 ('-', 'advice', {'weight': 1.0645282722332636}),
 ('-', 'advisors', {'weight': 2.3862841119626634}),
 ('-', 'alleviate', {'weight': 2.049811875378239}),
 ('-', 'amidst', {'weight': 2.3862841119626634}),
 ('-', 'animal', {'weight': 2.1631405606714464}),
 ('-', 'ap', {'weight': 1.693136931494689}),
 ('-', 'asked', {'weight': 1.826668324096219}),
 ('-', 'assessment', {'weight': 1.8754584882579868}),
 ('-', 'a

In [13]:
G_dict = load_pickle(os.path.join(data_dir,"text_graph.pkl"))
G = G_dict["graph"]
G

In [12]:
from scipy.sparse import coo_matrix
import numpy as np
matrix = np.matrix([[1, 2, 3], [3, 4, 1], [0, 1, 0]]) + np.eye(3)
display(matrix)
matrix = coo_matrix(matrix)
rowsum = np.array(matrix.sum(1))
display(rowsum)

matrix([[2., 2., 3.],
        [3., 5., 1.],
        [0., 1., 1.]])

array([[7.],
       [9.],
       [2.]])

In [16]:
import scipy.sparse as sp
d_inv_sqrt = np.power(rowsum, -0.5)
display(d_inv_sqrt)
display(d_inv_sqrt.flatten())
d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
# (A * D)T * D = DT * AT * D = DT * A * D
new_matrix = matrix.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()
new_matrix.todense()
# 1 X 3 --> 3 X 1

array([[0.37796447],
       [0.33333333],
       [0.70710678]])

array([0.37796447, 0.33333333, 0.70710678])

In [ ]:
matrix = np.matrix([[1, 2, 3], [3, 4, 1], [0, 1, 0]]) + np.eye(3)
# A = degree matrix
degrees = [0 if d == 0 else d[1]**(-0.5) for d in G.degree(weight = None)]
degrees = np.diag(degrees)
X = np.eye(G.number_of_nodes()) # features are just identity matrix
A_hat = degrees @ A @ degrees
f = X # (n X n) X (n X n) x (n X n) X (n X n) input of net
return A_hat, f

In [22]:
def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

def parse_index_file(filename):
    """Parse index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

old_model_datadir = os.path.abspath(os.path.join('..', '..','BGSRD', 'data'))
old_model_datadir

import pickle as pkl

In [61]:
names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'adj']
objects = []
dataset_str = 'botometer'
for i in range(len(names)):
    with open(os.path.join(old_model_datadir, "ind.{}.{}".format(dataset_str, names[i])), 'rb') as f:
        objects.append(pkl.load(f, encoding='latin1'))
x, y, tx, ty, allx, ally, adj = tuple(objects)

# compute number of real train/val/test/word nodes and number of classes
nb_node = adj.shape[0] # number of nodes in the graph - 3729
nb_train, nb_val, nb_test = train_mask.sum(), val_mask.sum(), test_mask.sum() # 302, 33, 111
nb_word = nb_node - nb_train - nb_val - nb_test # 3283
nb_class = y_train.shape[1] # number of classes



def encode_input(text, tokenizer):
    input = tokenizer(text, max_length=max_length, truncation=True, padding='max_length', return_tensors='pt')
#     print(input.keys())
    return input.input_ids, input.attention_mask


input_ids, attention_mask = encode_input(text, model.tokenizer)
input_ids = th.cat([input_ids[:-nb_test], th.zeros((nb_word, max_length), dtype=th.long), input_ids[-nb_test:]])
attention_mask = th.cat([attention_mask[:-nb_test], th.zeros((nb_word, max_length), dtype=th.long), attention_mask[-nb_test:]])

# transform one-hot label to class ID for pytorch computation
y = y_train + y_test + y_val
y_train = y_train.argmax(axis=1) # array with 1 if it's bot and 0 otherwise # 83
y = y.argmax(axis=1) # array with 1 if it's bot and 0 otherwise

# document mask used for update feature
doc_mask  = train_mask + val_mask + test_mask

# build DGL Graph
adj_norm = normalize_adj(adj + sp.eye(adj.shape[0]))
g = dgl.from_scipy(adj_norm.astype('float32'), eweight_name='edge_weight')
g.ndata['input_ids'], g.ndata['attention_mask'] = input_ids, attention_mask
g.ndata['label'], g.ndata['train'], g.ndata['val'], g.ndata['test'] = \
    th.LongTensor(y), th.FloatTensor(train_mask), th.FloatTensor(val_mask), th.FloatTensor(test_mask)
g.ndata['label_train'] = th.LongTensor(y_train)
g.ndata['cls_feats'] = th.zeros((nb_node, model.feat_dim))

logger.info('graph information:')
logger.info(str(g))

# create index loader
train_idx = Data.TensorDataset(th.arange(0, nb_train, dtype=th.long))
val_idx = Data.TensorDataset(th.arange(nb_train, nb_train + nb_val, dtype=th.long))
test_idx = Data.TensorDataset(th.arange(nb_node-nb_test, nb_node, dtype=th.long))
doc_idx = Data.ConcatDataset([train_idx, val_idx, test_idx])

idx_loader_train = Data.DataLoader(train_idx, batch_size=batch_size, shuffle=True)
idx_loader_val = Data.DataLoader(val_idx, batch_size=batch_size)
idx_loader_test = Data.DataLoader(test_idx, batch_size=batch_size)
idx_loader = Data.DataLoader(doc_idx, batch_size=batch_size, shuffle=True)

<302x300 sparse matrix of type '<class 'numpy.float64'>'
	with 90600 stored elements in Compressed Sparse Row format>

(302, 300) (302, 2) (111, 300) (111, 2) (3618, 300) (3618, 2) (3729, 3729)


C:\Users\nafta\AppData\Local\Temp\ipykernel_16036\683001919.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.array(mask, dtype=np.bool)


array([ True,  True,  True, ..., False, False, False])

3729

...

[0. 0.]
[0. 0.]
[1. 0.]


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

3729

[3,
 4,
 6,
 7,
 9,
 11,
 13,
 14,
 24,
 28,
 29,
 37,
 41,
 44,
 51,
 53,
 63,
 64,
 67,
 72,
 78,
 79,
 83,
 84,
 86,
 92,
 100,
 102,
 104,
 108,
 113,
 115,
 124,
 139,
 142,
 150,
 157,
 164,
 165,
 166,
 167,
 168,
 171,
 177,
 178,
 180,
 181,
 184,
 193,
 198,
 200,
 201,
 203,
 206,
 208,
 209,
 210,
 212,
 214,
 221,
 225,
 226,
 229,
 231,
 235,
 243,
 251,
 254,
 265,
 266,
 269,
 273,
 275,
 278,
 279,
 281,
 284,
 285,
 287,
 288,
 289,
 297,
 300]

83

In [6]:
import torch as th

df_text = load_pickle(os.path.join(data_dir, 'df_data.pkl'))
df_text['text'] = df_text['text'].apply(lambda x: ' '.join(x))
G_dict = load_pickle(os.path.join(data_dir, "text_graph.pkl"))
G = G_dict["graph"]

nb_node = G.number_of_nodes()
nb_train, nb_val, nb_test = len(df_text[df_text.type == 'train']), len(df_text[df_text.type == 'val']), len(df_text[df_text.type == 'test'])
nb_document = nb_train + nb_val + nb_test 
nb_word = nb_node - nb_document
nb_class = df_text['label'].nunique() # number of classes

nb_node = G.number_of_nodes()
nb_train, nb_val, nb_test = len(df_text[df_text.type == 'train']), len(df_text[df_text.type == 'val']), len(df_text[df_text.type == 'test'])
nb_document = nb_train + nb_val + nb_test 
nb_word = nb_node - nb_document
nb_class = df_text['label'].nunique() # number of classes


idx_train_real = df_text[(df_text.label == 1) & (df_text.type == 'train')].index.to_list()
idx_val_real = df_text[(df_text.label == 1) & (df_text.type == 'val')].index.to_list()
test_real_index = df_text[(df_text.label == 1) & (df_text.type == 'test')].index.to_list()
idx_test_real = list(map(lambda x: x + nb_word, test_real_index))

# 1d array with 1 if the index belong to train and label == 1
y_train, y_val, y_test = np.zeros(nb_node), np.zeros(nb_node), np.zeros(nb_node)
y_train[idx_train_real] = 1
y_val[idx_val_real] = 1
y_test[idx_test_real] = 1
y = th.LongTensor(y_train + y_val + y_test)



In [7]:
label = {}
label['train'], label['val'], label['test'] = y[:nb_train], y[nb_train:nb_train + nb_val], y[-nb_test:]

In [11]:
th.set_printoptions(profile = "full")
print("Train:")
print(list(th.Size(label['train'])))
print(len(list(th.Size(label['train']))))
print("Val:")
print(list(th.Size(label['val'])))
print(len(list(th.Size(label['val']))))
print("Test:")
print(list(th.Size(label['test'])))
print(len(list(th.Size(label['test']))))


Train:
[1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1]
288
Val:
[1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0]
32
Test:
[0, 1, 0, 1, 

In [13]:
print(len(list(th.Size(y))))


3376


In [ ]:
import torch.utils.data as Data


input_ids, attention_mask = {}, {}
input_ids_, attention_mask_ = encode_input(df_text.text.to_list(), model.tokenizer, max_length, True)

# create train/test/val datasets and dataloaders
input_ids['train'], input_ids['val'], input_ids['test'] =  input_ids_[:nb_train], input_ids_[nb_train:nb_train+nb_val], input_ids_[-nb_test:]
attention_mask['train'], attention_mask['val'], attention_mask['test'] =  attention_mask_[:nb_train], attention_mask_[nb_train:nb_train+nb_val], attention_mask_[-nb_test:]

datasets = {}
loader = {}
for split in ['train', 'val', 'test']:
    datasets[split] =  Data.TensorDataset(input_ids[split], attention_mask[split], label[split])
    loader[split] = Data.DataLoader(datasets[split], batch_size = 128, shuffle = True)